In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
bankruptcy_df = pd.read_csv(r'/content/gdrive/MyDrive/financial_data_mining/bankruptcy.csv',header =0)

In [3]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from random import seed
from random import random

In [4]:
seed(1)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 

X = bankruptcy_df[bankruptcy_df.columns.difference(['Firm', 'FAIL'])]
y = bankruptcy_df[['FAIL']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.feature_selection import SelectKBest
k = 10
prep = SelectKBest(k=k)
X_train = prep.fit_transform(X_train, y_train.values.flatten())
X_test = prep.fit_transform(X_test, y_test.values.flatten())

In [6]:
import csv
import numpy as np
import math
import cvxopt as opt
from cvxopt import matrix, solvers

In [7]:
a1 = X_train[:5]

In [13]:
k = np.zeros((5,5))
for i in range(5):
  for j in range(5):
    k[i,j] = (a1[i].dot(a1[j]) +1)**2

In [14]:
k

array([[11.9660511 ,  6.21890106, 19.16827646,  1.48092323,  0.15831272],
       [ 6.21890106, 61.50501275,  7.18455026,  0.13515003,  1.39615196],
       [19.16827646,  7.18455026, 49.20374661,  1.05930864,  2.02410659],
       [ 1.48092323,  0.13515003,  1.05930864,  7.18245891,  1.23073045],
       [ 0.15831272,  1.39615196,  2.02410659,  1.23073045, 39.33331164]])

In [15]:
(a1.dot(a1.T) + 1)**2

array([[11.9660511 ,  6.21890106, 19.16827646,  1.48092323,  0.15831272],
       [ 6.21890106, 61.50501275,  7.18455026,  0.13515003,  1.39615196],
       [19.16827646,  7.18455026, 49.20374661,  1.05930864,  2.02410659],
       [ 1.48092323,  0.13515003,  1.05930864,  7.18245891,  1.23073045],
       [ 0.15831272,  1.39615196,  2.02410659,  1.23073045, 39.33331164]])

In [ ]:
def gauss(x, y, sigma=.1):
  return np.exp((-np.linalg.norm(x - y, axis=1)**2) / (2*(sigma**2)))

In [37]:
k = np.zeros((5,5))

np.exp(-np.linalg.norm(a1[0] - a1, axis=1)**2 / (2*(0.1**2)))


array([1.00000000e+000, 7.75570046e-138, 5.09022716e-038, 3.46298227e-081,
       9.50051822e-195])

In [38]:
for i in range(5):
  k[i] = np.exp((-np.linalg.norm(a1[i] - a1, axis=1)**2) / (2*(.1**2)))
k

array([[1.00000000e+000, 7.75570046e-138, 5.09022716e-038,
        3.46298227e-081, 9.50051822e-195],
       [7.75570046e-138, 1.00000000e+000, 6.18828618e-207,
        2.96588148e-213, 6.77760801e-256],
       [5.09022716e-038, 6.18828618e-207, 1.00000000e+000,
        1.52864993e-166, 0.00000000e+000],
       [3.46298227e-081, 2.96588148e-213, 1.52864993e-166,
        1.00000000e+000, 6.27375394e-147],
       [9.50051822e-195, 6.77760801e-256, 0.00000000e+000,
        6.27375394e-147, 1.00000000e+000]])

In [169]:
def kernel(XTest, XTrain, ktype = 0, sigma = 0):
    # Write your code below
  """
  ktype=0 : RBF
  ktype=1 : Polynomial (sigma is the exponent)
  ktype=2 : Linear
  """

  m = XTest.shape[0]
  if ktype == 0:

    K = np.zeros((m,m))
    for i in range(m):
      
      K[i] = np.exp((-np.linalg.norm(XTest[i] - XTrain, axis=1)**2) / (2*(sigma**2)))

  elif ktype == 1:
    K = (1 + XTest.dot(XTrain.T))**sigma

  elif ktype == 2:
    K = XTest.dot(XTrain.T)

  return K   

def predict(XTest, XTrain, yTrain, alpha):
    # Write your code below 

  pred = None
  return pred

def intercept(XTest, XTrain, yTrain, alpha):
  


  return b


# Linear Kernel
# TrainSize = X_train.shape[0]
# K = kernel(X_train, X_train, 2, 2)
# a0 = opt.matrix(np.random.randn(TrainSize))

# # Inequality that individual alpha>=0
# G = opt.matrix(np.eye(TrainSize))
# h = opt.matrix(np.zeros(TrainSize))

# # Equality that sum(alpha_i*y_i)=0
# A = opt.matrix(np.double(y_train))
# b = opt.matrix(0.0)

# # Change from min to max optimization by multiplying with -1
# # Regularization term to force H positive definite
# ymat = np.diag(y_train.values.flatten())
P2 = 0.5 * np.dot(ymat, np.dot(K, ymat)) + 1e-10 * np.identity(TrainSize)
# q = opt.matrix(-np.ones((TrainSize,1)))

# opts = {'maxiters':1000}
# solvers.options['show_progress'] = False 
# sol = solvers.qp(opt.matrix(P), q, G, h, A.T, b, initvals=a0, options=opts) # solvers.qp to solve the optimization
# alpha = np.array(sol['x'])
m = X_train.shape[0]
K = kernel(X_train, X_train, 2, 2)
P = opt.matrix(np.outer(y_train, y_train) * K)
q = opt.matrix(-np.ones((m, 1)))
G = opt.matrix(np.vstack((np.eye(m) * -1, np.eye(m))))
h = opt.matrix(np.hstack((np.zeros(m), np.ones(m) * 1)))
A = opt.matrix(y_train.values, (1, m), "d")
b = opt.matrix(np.zeros(1))
opt.solvers.options["show_progress"] = False
sol = opt.solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol["x"])
(alphas > 0.0001).sum()

# pred = predict(X_test, X_train, y_train.values, alpha)
# print('Accuracy: %f\n' % (np.mean(pred == y_test) * 100))

27

In [188]:
support_vector = alphas > 0.0001
t_train = y_train.copy()
t_train[y_train == 0] = -1
t_train[y_train == 1] = 1
t_train = t_train.values

train_kernel = X_train[support_vector.reshape(48, )]
new_kernel = kernel(train_kernel[[0]], train_kernel, 2, 2)
b = np.zeros(new_kernel.shape[1])
for i in range(new_kernel.shape[1]):
   b[i] = t_train[i] - (alphas[support_vector, None] * t_train[support_vector] * kernel(train_kernel[[i]], train_kernel, 2, 2).reshape(27, -1)).sum()  
b.sum() / new_kernel.shape[1]


594.5401697500298

In [186]:
new_kernel.shape[1]

27